In [ ]:
import os
import sys
#sys.path.append('../')
import pandas as pd
import numpy as np
from statsmodels.stats.power import TTestIndPower
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import FTestAnovaPower
from src.data_quality_checks import check_missing_data
from scipy.stats import chi2_contingency

In [3]:
current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

NameError: name 'os' is not defined

Load data

In [1]:
# Loading historical data
file_path = "../data/MachineLearningRating_v3.txt"

df = pd.read_csv(file_path, sep='|', low_memory=False)

NameError: name 'pd' is not defined